

# Homework 2 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so there's no need to load it from disk.

In [2]:
import random
import string
import numpy as np
import torch
import torch.nn as nn
# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
    """ Converts the generated integers to one-hot vectors """
    ones = torch.sparse.torch.eye(k)
    shape = x.shape
    res = ones.index_select(0, x.view(-1).type(torch.int64))
    return res.view(*shape, res.shape[-1])


class EchoDataset(torch.utils.data.IterableDataset):

    def __init__(self, delay=4, seq_length=15, size=1000):
        self.delay = delay
        self.seq_length = seq_length
        self.size = size

    def __len__(self):
        return self.size

    def __iter__(self):
        """ Iterable dataset doesn't have to implement __getitem__.
            Instead, we only need to implement __iter__ to return
            an iterator (or generator).
        """
        for _ in range(self.size):
            seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
            result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
            yield seq, result

DELAY = 4
DATASET_SIZE = 200000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)

## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`.

In [3]:
class GRUMemory(nn.Module):

    def __init__(self, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size=27, hidden_size=hidden_size, num_layers = 3, batch_first=True)
        self.decoder = nn.Linear(hidden_size, 27)

    def forward(self, x):
        # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
        x = self.gru(x)[0]
        logits = self.decoder(x)
        return logits

    @torch.no_grad()
    def test_run(self, s):
        # This function accepts one string s containing lowercase characters a-z.
        # You need to map those characters to one-hot encodings,
        # then get the result from your network, and then convert the output
        # back to a string of the same length, with 0 mapped to ' ',
        # and 1-26 mapped to a-z.
        self.eval()
        s_to_idx = torch.tensor([ord(i) - ord('a') + 1 for i in s])
        x = idx_to_onehot(s_to_idx).unsqueeze(0).to(device)
        result = torch.argmax(self.forward(x),2)
        
        s_out = ''
        for c in result[0]:
            if c == 0:
                s_out += ' '
            else:
                s_out += chr(ord('a') + c - 1)
        return s_out

# gru_memory = GRUMemory(hidden_size=64)
# s = 'abacaba'
# out = gru_memory.test_run(s)
# print("out)

## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [4]:
def test_model(model, sequence_length=15):
    """
    This is the test function that runs 100 different strings through your model,
    and checks the error rate.
    """
    total = 0
    correct = 0
    for i in range(500):
        s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
        result = model.test_run(s)
        for c1, c2 in zip(s[:-DELAY], result[DELAY:]):
            correct += int(c1 == c2)
        total += len(s) - DELAY

    return correct / total

In [5]:
model = GRUMemory(64)
train_loader = torch.utils.data.DataLoader(ds, batch_size=64)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 4

if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

In [190]:
import time
start_time = time.time()

# TODO
for epoch in range(1,num_epochs+1):
    model.train()
    
    for input, target in train_loader:
        input = idx_to_onehot(input).to(device)
        target_onehot = idx_to_onehot(target).to(device)
        out = model(input)
        loss = criterion(out[:,DELAY:,:], target_onehot[:,DELAY:,:])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Train Epoch: {epoch}/{num_epochs}, loss: {loss:.3f}')
end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

Train Epoch: 1/4, loss: 0.105
Train Epoch: 2/4, loss: 0.103
Train Epoch: 3/4, loss: 0.098
Train Epoch: 4/4, loss: 0.098
tests passed


## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay is constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [6]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

    def __init__(self, max_delay=8, seq_length=20, size=1000):
        self.max_delay = max_delay
        self.seq_length = seq_length
        self.size = size

    def __len__(self):
        return self.size

    def __iter__(self):
        for _ in range(self.size):
            seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
            delay = random.randint(0, self.max_delay)
            result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
            yield seq, delay, result

### Model

And the model.

In [7]:
class VariableDelayGRUMemory(torch.nn.Module):

    def __init__(self, hidden_size, max_delay, device=torch.device("cpu")):
        super().__init__()
        #TODO
        self.hidden_size = hidden_size
        self.max_delay = max_delay
        self.device = device
        self.linear = torch.nn.Linear(1, hidden_size)
        
        self.gru = torch.nn.GRU(
        input_size=27,
        hidden_size=self.hidden_size,
        num_layers=max_delay,
        batch_first=True,
        )
        self.decoder = torch.nn.Linear(
            in_features=hidden_size,
            out_features=27
        )
    def forward(self, x, delays):
        # inputs:
        # x - tensor of shape (batch size, seq length, N + 1)
        # delays - tensor of shape (batch size)
        # returns:
        # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
        # TODO
        hidden0 = self.linear(delays.view(-1,1)).unsqueeze(0).repeat(8,1,1)
        x, hidden = self.gru(x, hidden0.to(device))

        x = self.decoder(x)
        x = torch.nn.functional.log_softmax(x, dim=2)
        
        return x, hidden
    
    @torch.no_grad()
    def test_run(self, s, delay):
        # This function accepts one string s containing lowercase characters a-z,
        # and a delay - the desired output delay.
        # You need to map those characters to one-hot encodings,
        # then get the result from your network, and then convert the output
        # back to a string of the same length, with 0 mapped to ' ',
        # and 1-26 mapped to a-z.

        # TODO
        self.eval()
        s_to_idx = torch.tensor([ord(i) - ord('a') + 1 for i in s])
        x = idx_to_onehot(s_to_idx).float().unsqueeze(0).to(self.device)
        
        delays = torch.tensor([delay]).float().to(self.device)

        result, _ = self.forward(x, delays)
        result = result.squeeze()
        out = torch.max(result,1)
        s_out = ''
        for c in out.indices:
            if c == 0:
                s_out += ' '
            else:
                s_out += chr(ord('a') + c - 1)
        return s_out

In [8]:
# from torchinfo import summary
# model = VariableDelayGRUMemory(64,8)
# summary(model,[(64,20,27),[64]])

Layer (type:depth-idx)                   Output Shape              Param #
VariableDelayGRUMemory                   [64, 20, 27]              --
├─Linear: 1-1                            [64, 64]                  128
├─GRU: 1-2                               [64, 20, 64]              192,576
├─Linear: 1-3                            [64, 20, 27]              1,755
Total params: 194,459
Trainable params: 194,459
Non-trainable params: 0
Total mult-adds (M): 246.62
Input size (MB): 0.14
Forward/backward pass size (MB): 0.96
Params size (MB): 0.78
Estimated Total Size (MB): 1.88

### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [9]:
def test_variable_delay_model(model, seq_length=20):
    """
    This is the test function that runs 100 different strings through your model,
    and checks the error rate.
    """
    total = 0
    correct = 0
    for i in range(500):
        s = ''.join([random.choice(string.ascii_lowercase) for i in range(seq_length)])
        d = random.randint(0, model.max_delay)
        result = model.test_run(s, d)
        if d > 0:
            z = zip(s[:-d], result[d:])
        else:
            z = zip(s, result)
        for c1, c2 in z:
            correct += int(c1 == c2)
        total += len(s) - d

    return correct / total

In [13]:
ds = VariableDelayEchoDataset(max_delay=8, seq_length=20, size=DATASET_SIZE)
train_dataloader = torch.utils.data.DataLoader(ds, batch_size=64, num_workers=2)

model = VariableDelayGRUMemory(64, 8)
criterion = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    device = torch.device("cuda:0")
    model.device = device
else:
    device = torch.device("cpu")

def train(model, train_dataloader):
    model.train()

    for input, delays, target in train_dataloader:
        input = idx_to_onehot(input).to(device)
        delays = delays.to(device)
        target = target.to(device)
        
        optimizer.zero_grad()
        out, hidden = model(input.float(), delays.float())
        out = out.permute(0,2,1)
        
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()

    return loss, hidden

In [14]:
import time
start_time = time.time()

MAX_DELAY = 8
SEQ_LENGTH = 20

# TODO: implement model training here.

n_epochs = 4

for epoch in range(1, n_epochs+1):
    loss, _ = train(model, train_dataloader)
    print(f'Train Epoch: {epoch}/{n_epochs}, loss: {loss:.3f}')

end_time = time.time()
assert end_time - start_time < 600, 'executing took longer than 10 mins'
assert test_variable_delay_model(model) > 0.99, 'accuracy is too low'
print('tests passed')

Train Epoch: 1/4, loss: 0.215
Train Epoch: 2/4, loss: 0.011
Train Epoch: 3/4, loss: 0.006
Train Epoch: 4/4, loss: 0.006
tests passed
